# Assignment 2: spectral graph theory and graph signal processing

**Deadline: April 24th**

This assignment features three exercices: 
  - label propagation
  - spectral clustering
  - spectral graph filtering

We will need a package called [basemap](https://basemaptutorial.readthedocs.io/en/latest/) that provides a map of earth. This package can be installed in a conda environment by running:

`conda install -c conda-forge basemap`


If you don't manage to install this package, simply print all the plots without the background.

After installing the package, the following lines may be needed:

```
import os
import conda
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib
```


## Exercice 1: Spectral clustering on air routes

In this exercice, we will perform spectral clustering on the flight routes network and analyze the influence of the graph on the obtained clusters.

*For additional information on spectral clustering, we refer to this [tutorial paper](http://www.tml.cs.uni-tuebingen.de/team/luxburg/publications/Luxburg07_tutorial.pdf). For practical aspects of the algorithm, the relevant sections are 1, 2, 3, 4, 8.*

### Load and visualize data

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap
from sklearn.cluster import KMeans

In [ ]:
airports = pd.read_csv("./airports_cleaned.csv")
dists_df = pd.read_csv("./dists.csv")

# Create graph from distances dataframe
G_full = nx.from_pandas_edgelist(dists_df,  "source_airport_id", "destination_airport_id", ["distance"])

airport_ids_full = airports['airport_id'].to_numpy()

lat_full = airports['Latitude'].to_numpy()[:, np.newaxis]
long_full = airports['Longitude'].to_numpy()[:, np.newaxis]

n_clusters = 5

In [ ]:
dists_df

In [ ]:
airports.head()

In [ ]:
def plot_map(longitude, latitude, predictions=None, title=None):
    """ longitude: np.array
        latitude: np.array of the same length
        predictions (optional): numpy array of int corresponding to cluster assignment. """
    plt.figure(figsize=(12,6))
    m=Basemap()
    m.drawcoastlines()                                     # Show the coast lines
    plt.scatter(longitude, latitude, c=predictions)
    if title is not None:
        plt.title(title)
    plt.show()


plot_map(long_full, lat_full, title="All airports")

### Part 1: K-means

**Question 1 (code): Run K-means with 5 clusters on this data.**

You can use the scikit learn function (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

In [ ]:
# Write your code here
y_pred_k_means= None

In [ ]:
plot_map(long_full, lat_full, predictions=y_pred_k_means, title=f"K means with {n_clusters} clusters")

**Question 2 (analysis): Why is the standard implementation of K-means not suited to data that lies on a sphere?**

*Write your answer here*

### Part 2: Spectral clustering

#### Data preprocessing
The flight routes network is not connected. We will select the largest connected component and discard the rest.

In [ ]:
def largest_component(G: nx.Graph):
    """ Return the ids of the airports inside the largest connected component.
        Input: G: networkx graph with n nodes
        Returns: connected_airport_ids: list of int
    """
    largest_connected_component = max(nx.algorithms.components.connected_components(G), key=len)
    return G.subgraph(largest_connected_component)


G = largest_component(G_full)
mask = np.zeros(len(airport_ids_full), dtype=bool)

for node in G.nodes:
    row = np.nonzero(node == airport_ids_full)
    if len(row) != 1:
        raise ValueError(f"Issue with node {node}: nonzero returns {row}")
    mask[row] = True
    
long = long_full[mask]
lat = lat_full[mask]
airport_ids = airport_ids_full[mask]
A = nx.adjacency_matrix(G).toarray().astype(float)

**Question 3 (code): Fill `compute_laplacian`, `compute_number_connected_components`, `spectral decomposition`. These functions should work for any definition of the laplacian (combinatorial, symmetric normalized, random walk).**

Warning: the eigendecomposition of a non symmetric matrix returns complex numbers, even if the imaginary part is in fact 0.

In [ ]:
def compute_laplacian(adjacency: np.ndarray, normalize: str):
    """ normalize: can be None, 'sym' or 'rw' for the combinatorial, symmetric normalized or random walk Laplacians
    Return:
        L (n x n ndarray): combinatorial or symmetric normalized Laplacian.
    """
    pass


def compute_number_connected_components(lamb: np.array, threshold: float):
    """ lamb: array of eigenvalues of a Laplacian
        Return:
        n_components (int): number of connected components.
    """
    pass


def spectral_decomposition(laplacian: np.ndarray):
    """ Warning: the choice of the solver should depends on whether the laplacian is symmetric.
        Return:
        lamb (np.array): eigenvalues of the Laplacian
        U (np.ndarray): corresponding eigenvectors.
    """
    pass

In [ ]:
laplacian_comb = compute_laplacian(A, normalize=None)
lamb_comb, U_comb = spectral_decomposition(laplacian_comb)

laplacian_norm = compute_laplacian(A, normalize='sym')
lamb_norm, U_norm = spectral_decomposition(laplacian_norm)

laplacian_rw = compute_laplacian(A, normalize='rw')
lamb_rw, U_rw = spectral_decomposition(laplacian_rw)
lamb_rw = np.real(lamb_rw)

**Question 4 (code): Implement spectral clustering for the 3 possible definitions of the Laplacian.**

In [ ]:
class SpectralClustering():
    def __init__(self, n_classes: int, normalize: str):
        self.n_classes = n_classes
        self.normalize = normalize
        self.clustering_method = None        # To fill
        
    def fit_predict(self, adjacency):
        """ Your code should be correct both for the combinatorial
            and the symmetric normalized spectral clustering.
            Return:
            y_pred (np.ndarray): cluster assignments.
        """
        pass

#### Case 1: Spectral clustering with the flight route unweighted adjacency matrix

In [ ]:
spectral_clustering = SpectralClustering(n_classes=4, normalize=None)
y_pred = spectral_clustering.fit_predict(A)

spectral_clustering_sym = SpectralClustering(n_classes=4, normalize='sym')
y_pred_sym = spectral_clustering_sym.fit_predict(A.astype(float))

spectral_clustering_rw = SpectralClustering(n_classes=4, normalize='rw')
y_pred_rw = spectral_clustering_rw.fit_predict(A.astype(float))

In [ ]:
plot_map(long, lat, predictions=y_pred,
         title="Spectral clustering on A with the combinatorial Laplacian")

plot_map(long, lat, predictions=y_pred_sym,
         title="Spectral clustering on A with the symmetric normalized Laplacian")

plot_map(long, lat, predictions=y_pred_rw,
         title="Spectral clustering on A with the random walk Laplacian")

Note that imbalanced clusters may not necessarily be due to a bug. In spectral clustering, there is no easy way to ensure that the predicted classes are balanced.

**Question 5 (analysis): Are some clusters interpretable? Why don't we get something similar to K-means (where clusters approximately correspond to continents)?**

*your answer here*

#### Spectral clustering on a distance-weighted matrix

We will also run spectral clustering on a second adjacency matrix, where short flight routes have a larger edge weights that long routes. The following lines build the new adjacency matrix W.

In [ ]:
routes_distance_matrix = nx.adjacency_matrix(G, weight='distance').toarray()     

matrix_mask = routes_distance_matrix > 0

average_distance = np.mean(routes_distance_matrix[matrix_mask])
sigma = 0.5 * average_distance                                    # width of the Gaussian kernel

W = np.zeros(routes_distance_matrix.shape)
W[matrix_mask] = np.exp(- routes_distance_matrix ** 2/ (2 * sigma ** 2))[matrix_mask] # Gaussian kernel
W[matrix_mask * W < 0.0001] = 0.0001                       # Used to keep the graph connected

In [ ]:
spectral_clustering = SpectralClustering(n_classes=6, normalize=None)
y_pred = spectral_clustering.fit_predict(W)

spectral_clustering_sym = SpectralClustering(n_classes=6, normalize='sym')
y_pred_sym = spectral_clustering_sym.fit_predict(W)

spectral_clustering_rw = SpectralClustering(n_classes=6, normalize='rw')
y_pred_rw = spectral_clustering_rw.fit_predict(W)


In [ ]:
plot_map(long, lat, predictions=y_pred,
         title="Spectral clustering on W with the combinatorial Laplacian")

plot_map(long, lat, predictions=y_pred_sym,
         title="Spectral clustering on W with the symmetric normalized Laplacian")

plot_map(long, lat, predictions=y_pred_rw,
         title="Spectral clustering on W with the random walk Laplacian")

**Question 6 (analysis): Are some clusters interpretable? How do the results differ from the ones obtained with the unweighted adjacency matrix A?**

*your answer here*


# Exercice 2: spectral filtering

This exercice covers basic notion of filtering in the graph Fourier domain. We will use a nearest-neighbor graph constructed from the Stanford Bunny point cloud included in the PyGSP library.
We construct the normalized graph laplacians from the adjacency matrix and find its spectral decomposition.

In [ ]:
import pygsp
from pygsp.graphs import Bunny
from mpl_toolkits.mplot3d import Axes3D # as Axes3d

In [ ]:
# Define the graph
G = Bunny()
adjacency = np.asarray(G.W.todense())
n_nodes = adjacency.shape[0]


def plot_bunny(x=None, title='', vlim=[-0.03, 0.03]):
    """ PLot a signal x on the bunny graph. """
    fig = plt.gcf()
    ax = plt.gca()
    if not isinstance(ax, Axes3D):
        ax = plt.subplot(111, projection='3d')
    if x is not None:
        x = np.squeeze(x)
    p = ax.scatter(G.coords[:,0], G.coords[:,1], G.coords[:,2], c=x, marker='o',
                   s=5, cmap='RdBu_r', vmin=vlim[0], vmax=vlim[1])
    ax.view_init(elev=-90, azim=90)
    ax.dist = 7
    ax.set_axis_off()
    ax.set_title(title)
    if x is not None:
        fig.colorbar(p)
    plt.show()
        
        
plt.subplot(111, projection='3d')
plot_bunny()

# Compute the normalized Laplacian
laplacian = compute_laplacian(adjacency, normalize='sym')
laplacian = (laplacian + laplacian.T) / 2    # Make sure the matrix is symmetric despite numerical errors
lam, U = spectral_decomposition(laplacian)

# This lines should normally not be needed, but it seems that not all eigenvalues are always sorted
argsort = np.argsort(lam)
lam = lam[argsort]
U = U[:, argsort]

# Eigenvalues plot
plt.figure(figsize=(6, 5))
plt.scatter(np.arange(len(lam)), lam)
plt.title('Eigenvalues of $L_{norm}$')
plt.show()


# Plot some eigenvectors of the graph
plt.figure(figsize=(18, 9))
plt.subplot(231, projection='3d')
plot_bunny(x=U[:,0], title='Eigenvector #0')
plt.subplot(232, projection='3d')
plot_bunny(x=U[:,1], title='Eigenvector #1')
plt.subplot(233, projection='3d')
plot_bunny(x=U[:,2], title='Eigenvector #2')

plt.subplot(234, projection='3d')
plot_bunny(x=U[:,3], title='Eigenvector #3')
plt.subplot(235, projection='3d')
plot_bunny(x=U[:,10], title='Eigenvector #10')
plt.subplot(236, projection='3d')
plot_bunny(x=U[:,100], title='Eigenvector #100')

**Question 7 (analysis): How does the intuitive notion of "smoothness" relates to the eigenvalue corresponding to an eigenvector? How can the smoothness of a signal be measured?**

*your answer here*

**Question 8 (code): Implement the Graph Fourier Transform (GFT) of a graph signal and its inverse.**

In [ ]:
def GFT(signal: np.array, U: np.ndarray):
    """ signal: float array of size n.
        U: matrix of size n x n containing one eigenvector per column.
    """
    pass

def iGFT(fourier_coefficients: np.ndarray, U: np.ndarray):
    """ fourier_coefficients: float array of size n, containing a signal represented in the spectral domain
        U: matrix of size n x n containing one eigenvector per column.
    """
    pass

### Filtering
We first define a signal `x`, and a noisy version of it `x_noisy`.

In [ ]:
x = G.coords[:, 0] + G.coords[:, 1] + 3 * G.coords[:, 2]
x /= np.linalg.norm(x) 

noise = np.random.randn(n_nodes)
noise /= np.linalg.norm(noise) 

x_noisy = x + 0.3 * noise

plot_bunny(x_noisy, vlim=[min(x_noisy), max(x_noisy)])

We will try to extract the signal from the noise using graph filters. Let us start by creating 4 ideal graph filters: a low pass, a band pass, a high pass, and the filter that implements the solution of Tikhonov regularization.

In [ ]:
ideal_lp = np.ones((n_nodes,))
ideal_bp = np.ones((n_nodes,))
ideal_hp = np.ones((n_nodes,))

ideal_lp[lam >= 0.1] = 0  # Low-pass filter with cut-off at lambda=0.1
ideal_bp[lam < 0.1] = 0  # Band-pass filter with cut-offs at lambda=0.1 and lambda=0.5
ideal_bp[lam > 0.5] = 0
ideal_hp[lam <= 1] = 0  # High-pass filter with cut-off at lambda=1

alpha = 0.99 / np.max(lam)

ideal_tk = np.ones((n_nodes,))
ideal_tk = 1 / (1 + alpha*lam)

# Let's plot the spectral responses:

plt.plot(lam, ideal_lp, '-', label='LP')
plt.plot(lam, ideal_bp, '-', label='BP')
plt.plot(lam, ideal_hp, '-', label='HP')
plt.plot(lam, ideal_tk, '-', label='Tikhonov')
plt.xlabel('$\lambda$')
plt.ylabel('Spectral response')
plt.legend(loc='lower right')
plt.show()

**Question 9 (code): Implement the filtering function**

In [ ]:
def filter_signal(x: np.array, spectral_response: np.array, U: np.ndarray):
    """ Return a filtered signal
        The filter is defined in the spectral domain by its value on each eigenvector
        x (float array of size n): input signal
        spectral response (float array of size n): value of the filter at each eigenvalue
        U (n x n matrix): eigenvectors (one per column).
        returns:
        out (float array of size n): Filtered signal
    """
    pass

In [ ]:
x_lp = filter_signal(x_noisy,ideal_lp, U)
x_bp = filter_signal(x_noisy,ideal_bp, U)
x_hp = filter_signal(x_noisy,ideal_hp, U)
x_tk = filter_signal(x_noisy,ideal_tk, U)

plt.figure(figsize=(18, 9))
plt.subplot(231, projection='3d')
plot_bunny(x=x, title='signal (true)', vlim=[min(x), max(x)])
plt.subplot(232, projection='3d')
plot_bunny(x=x_noisy, title='signal (noisy)', vlim=[min(x), max(x)])
plt.subplot(233, projection='3d')
plot_bunny(x=x_lp, title='Low-pass', vlim=[min(x_lp), max(x_lp)])
plt.subplot(234, projection='3d')
plot_bunny(x=x_bp, title='Band-pass', vlim=[min(x_bp), max(x_bp)])
plt.subplot(235, projection='3d')
plot_bunny(x=x_hp, title='High-pass', vlim=[min(x_hp), max(x_hp)])
plt.subplot(236, projection='3d')
plot_bunny(x=x_tk, title='Tikhonov denoised signal', vlim=[min(x_tk), max(x_tk)])

**Question 10 (analysis): What kind of filter does Tikhonov regularization implement?**

*Your answer here*

## Exercice 3: Label propagation

We will implement label propagation, which is an algorithm for semi-supervised learning: given a few labeled points, the goal is to assign labels to other points. We refer to [this paper](http://mlg.eng.cam.ac.uk/zoubin/papers/CMU-CALD-02-107.pdf) for the details about the algorithm.

In [ ]:
import pygsp.graphs
import numpy as np
import numpy.linalg as LA
import numpy.random as npr
import matplotlib.pyplot as plt

We will use points sampled from the Swiss roll graph. Note that the (weighted) adjacency matrix **A** is already provided. If it was not the case, we would need to build a similarity graph (using nearest neighbors or an epsilon-neighborhood).

In [ ]:
n = 200


def sample_signal(n, num_classes=3, num_labels_per_class=4, seed=0):
    npr.seed(seed)
    indices = npr.choice(n,
                         size= num_labels_per_class * num_classes,
                         replace=False)
    indices = indices.reshape(num_labels_per_class, num_classes)

    x = np.zeros((n, num_classes))
    for j in range(num_classes):
        x[indices[:, j], j] = 1
    return x


swiss_roll = pygsp.graphs.SwissRoll(n, dim=2, noise=0.05)
A = swiss_roll.W                    # weighted adjacency matrix
coords = swiss_roll.coords          # coordinates
x = sample_signal(n)

swiss_roll.plot()

plt.figure()
c=np.argmax(x, axis=1)
plt.scatter(coords[:, 0], coords[:, 1], c=np.argmax(x, axis=1))
plt.scatter(coords[:, 0], coords[:, 1], c=x)
plt.title("Initial labels")
plt.show()

**Question 11 (code): Go through the paper and implement the label propagation algorithm of section 2.2**

In [ ]:
class LabelPropagation():
    def __init__(self, num_iters, stopping_criterion=None):
        self.num_iters = num_iters
        self.stopping_criterion = stopping_criterion
    
    def __call__(self, A, x, coords, plot_each_iteration=True):
        """ A: n x n: weighted adjacency matrix
            x: n x d matrix containing the initial labels
            coords: n x 2 matrix containing the point coordinates.
            plot_each_iteration: bool.
            Return:
            pred: n x d prediction matrix (each row should sum to 1)
        """
        plt.figure()
        plt.scatter(coords[:, 0], coords[:, 1], c=x)
        plt.title(f"Initial state")
        plt.show()
        
        # write your code here

        for i in range(self.num_iters):
            
            # write your code here
                
            if plot_each_iteration:
                plt.figure()
                plt.scatter(coords[:, 0], coords[:, 1], c=x)
                plt.title(f"After Iteration {i}")
                plt.show()
                
            if self.stopping_criterion is not None:
                if self.stopping_criterion.step(x):
                    print(f"Stopping criterion triggered after {i} iterations.")
                    break
        return x
    
    
def plot_predictions(x, coords):
    """ x: n x d matrix containing the predictions """
    predictions = np.argmax(x, axis=1)
    y = np.zeros(x.shape)
    for i in range(x.shape[0]):
        y[i, predictions[i]] = 1
    plt.figure()
    plt.scatter(coords[:, 0], coords[:, 1], c=y)
    plt.title("Final predictions")
    plt.show()

In [ ]:
num_iters = 20
lp = LabelPropagation(num_iters)
predictions = lp(A, x, coords)
plot_predictions(predictions, coords)

We will add a stopping criterion to our algorithm, and stop training when the updates of all points $i$ satisfy $||x_i^{t+1} - x_i^t||_2 \leq \epsilon$.

**Question 13 (code): Implement the uniform variation criterion.** *You can use the function `scipy.linalg.norm`*.

Tip: don't forget that unexpected behaviours can happen when arrays are modified in place. To avoid this, you can use `np.copy()`.

In [ ]:
class UniformVariationCriterion():
    def __init__(self, threshold=0.01):
        self.threshold = threshold
        self.last_values = None
        
    def step(self, x):
        """ Return True if the class probabilities for each point have changed by less than epsilon,
            False otherwise.
        """
        pass

In [ ]:
stopping_criterion = UniformVariationCriterion()
num_iters = 200

x = sample_signal(n)
lp = LabelPropagation(num_iters, stopping_criterion)
predictions = lp(A, x, coords, plot_each_iteration=False)
plot_predictions(predictions, coords)

An alternative way to perform semi-supervised learning is to apply a low-pass filter on the signal. Use the functions defined in the previous exercices to apply a Tikhonov filter with $\alpha=1$ to the signal `x`.

**Question 14 (code): Connection with low-pass filtering. Use the functions defined in the previous exercices to compute a Tikhonov filter. Apply this filter to the signal `x` to compute predictions for unseen nodes.**


In [ ]:
# Your code here
filtered = None

In [ ]:
plot_predictions(filtered, coords)